In [1]:
import torch 
from torch import nn
from torch import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)



cuda


In [2]:
class My_CBOW(nn.Module):
    def __init__(self, vob_size:int, embed_dim:int):
        super(My_CBOW,self).__init__()
        self.inEmbedding = nn.Embedding(vob_size,embed_dim)
        self.outEmbedding = nn.Embedding(vob_size,embed_dim)

    #    self.h = None
        
    
    def makeH(self,input):
        W = self.inEmbedding(input)
        n = W.shape[-2]

        h = torch.sum(W,dim=-2) / n
        return h

    def forwardWithH(self,h,out):
        W = self.outEmbedding(out)
        score = torch.sum(h * W,dim = -1)
        return score



# ## 模型测试

# cbow = My_CBOW(10,3)
# cbow.to(device=device)
# input = torch.Tensor([[1,2,5],[1,2,3],[6,2,3],[6,2,3]]).to(dtype=torch.int32,device = device)

# ##正样本
# h = cbow.makeH(input)
# pout = Tensor([1,3,2,5]).to(device=device,dtype=int)
# plable = torch.ones(pout.shape)
# pscore = cbow.forwardWithH(h,pout)


# ##负样本
# nout = Tensor([1,0,1,7]).to(device=device,dtype=int)
# nlable = torch.zeros_like(nout)
# nsocre = cbow.forwardWithH(h,nout)


# score = torch.cat((pscore,nsocre))
# lables = torch.cat((plable,nlable))

# print(score)



# loss_fn = torch.nn.BCEWithLogitsLoss()
# loss = loss_fn(score,lables)

# print(loss)

# loss.backward()

# # input = Tensor([[1,2,5],[1,2,3],[6,2,3]],dtype=torch.int)
# # print(input)

# # cbow.makeH(input)



In [12]:
import sys

sys.path.append("/content/drive/MyDrive/ml/nlp/")

import numpy
import pickle


from common.util import create_contexts_target
from dataset import ptb


window_size = 5
hidden_size =  100

batch_size = 100
max_epoch = 10


corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)

context , target = create_contexts_target(corpus=corpus,window_size=window_size)

print(context)
print(target)



[[   0    1    2 ...    8    9   10]
 [   1    2    3 ...    9   10   11]
 [   2    3    4 ...   10   11   12]
 ...
 [ 706 9999  119 ...  258   64   39]
 [9999  119 1143 ...   64   39   26]
 [ 119 1143   69 ...   39   26   24]]
[   5    6    7 ...  552  917 3196]


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
def train(model:My_CBOW, loss_fn , optimizer:torch.optim.Optimizer ,context : Tensor, target: Tensor, negativeTarget: Tensor):
    # model.train()
    h = model.makeH(context)

    p_lable = torch.ones_like(target).to(device=device)
    

    p_score = model.forwardWithH(h,target).to(device=device)

    n_lable = torch.zeros_like(negativeTarget).to(device=device)
    n_score = model.forwardWithH(h,negativeTarget).to(device=device)

    score = torch.cat((p_score,n_score)).to(dtype=torch.float32,device=device)
    lables = torch.cat((p_lable,n_lable)).to(dtype=torch.float32,device=device)

    loss = loss_fn(score,lables)
    loss.backward()
    optimizer.zero_grad()
    optimizer.step()
    return loss

    



In [14]:
import collections
import numpy as np
GPU = device == "cuda"
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # 在用GPU(cupy）计算时，优先速度
            # 有时目标词存在于负例中
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [18]:
train_context = torch.tensor(context).to(device=device)
train_target = torch.tensor(target).to(device = device)

# print(train_context.shape)
# print(train_target.shape)

# model = My_CBOW(vocab_size,hidden_size)
# optimizer = torch.optim.Adam(model.parameters())
# loss_fn = torch.nn.BCEWithLogitsLoss()




# ## 取十条负样本
# #sampler = torch.utils.data.Sampler(data_source)

# smapler = UnigramSampler(corpus,0.75,1)


# print(target.shape)


# batch_context = train_context[:10]
# batch_target = train_target[:10]



# negative = smapler.get_negative_sample(batch_target)
# print(negative.shape)

# negative = torch.from_numpy(negative).squeeze(dim=1)
# ##model.train()

# print(f"batch_target_shape{batch_target.shape}")
# train(model,loss_fn,optimizer,batch_context,batch_target,negative)





In [21]:
 
 
smapler = UnigramSampler(corpus,0.75,1)
loop = train_target.shape[0] // batch_size


print(loop)


model = My_CBOW(vocab_size,hidden_size)
model = model.to(device=device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.BCEWithLogitsLoss()


loss = None

for epoch in range(max_epoch):
    for i in range(loop):
        start = i * batch_size
        end = (i + 1) * batch_size

        b_context = train_context[start : end]
        b_target =  train_target[start : end]
        negative = smapler.get_negative_sample(b_target)
        negative = torch.from_numpy(negative).squeeze(dim=1).to(device= device)

        model.train()
        loss = train(model,loss_fn,optimizer,b_context,b_target,negative)

    print(f"{epoch} / {max_epoch} : losss is {loss}")

                

9295
0 / 10 : losss is 1.5197619199752808
1 / 10 : losss is 1.4933115243911743
2 / 10 : losss is 1.3742077350616455
3 / 10 : losss is 1.443870186805725
4 / 10 : losss is 1.3027026653289795
5 / 10 : losss is 1.5266680717468262
6 / 10 : losss is 1.6435636281967163
7 / 10 : losss is 1.4655804634094238
8 / 10 : losss is 1.6358784437179565
9 / 10 : losss is 1.4294499158859253


In [34]:
from common.util import most_similar


## CBOW 模型的评价

print(model.inEmbedding.weight.shape)


word_vecs = model.inEmbedding.weight.detach().to(device = "cpu").numpy()

querys = ['you','year','car','toyota']

for query in querys:
  most_similar(query,word_to_id,id_to_word,word_vecs)




torch.Size([10000, 100])

[query] you
 clues: 0.3754046857357025
 number: 0.34034666419029236
 am: 0.3396323025226593
 lipper: 0.33467087149620056
 disclosure: 0.32848480343818665

[query] year
 exploring: 0.35146403312683105
 ortiz: 0.34905633330345154
 monitors: 0.3476950526237488
 disrupted: 0.33855801820755005
 writing: 0.32244551181793213

[query] car
 adversary: 0.37265142798423767
 undeveloped: 0.34404247999191284
 midwestern: 0.32540544867515564
 realities: 0.3242040276527405
 verge: 0.31838104128837585

[query] toyota
 independence: 0.4084080755710602
 shot: 0.3998439610004425
 mushrooms: 0.38378170132637024
 haven: 0.33293184638023376
 trend: 0.3326318860054016
